In [50]:
import pandas as pd
import numpy as np
from langdetect import detect
from google_play_scraper import Sort, reviews

### ID von App aus dem Playstore
https://play.google.com/store/apps/details?id=com.sec.android.app.shealth&gl=DE 
Die entsprechende ID lautet com.sec.android.app.shealth

In [51]:
apps = {
    'Samsung Health':'com.sec.android.app.shealth',
    'MyFitnessPal: Kalorienzähler':'com.myfitnesspal.android',
    'WW (Weight Watchers)':'com.weightwatchers.mobile',
    'Kalorienzähler von FatSecret':'com.fatsecret.android',
    'YAZIO: Kalorienzähler & Fasten':'com.yazio.android',
    'HealthifyMe - Calorie Counter':'com.healthifyme.basic',
    'Lifesum: Gesunde Ernährung':'com.sillens.shapeupclub',
    'Noom: Gewicht & Health':'com.wsl.noom',
    'BetterMe: Gesundheits-Coaching':'com.gen.workoutme',
    'Lose It! - Kalorienzähler':'com.fitnow.loseit',
    'Kalorienzähler - EasyFit':'com.marioherzberg.swipeviews_tutorial1',
    'Kalorien, Fett & Eiweißzähler':'digifit.virtuagym.foodtracker',
    'Fitatu Kalorienzähler & Diät':'com.fitatu.tracker',
    'Gesundheit, Ernährung, Fitness- Kalorienzähler':'com.droidinfinity.healthplus',
    'Ernährungstagebuch':'com.dailybits.foodjournal',
    'Kalorienzähler HiKi':'ru.hikisoft.calories',
    'Calorie Counter - MyNetDiary':'com.fourtechnologies.mynetdiary.ad',
    'MyPlate Calorie Tracker':'com.livestrong.tracker',
    'Fddb - Kalorienzähler & Diät':'com.fddb',
    'Kalorienzähler':'com.nutratech.app.android',
    'Cronometer - CalorieTracker':'com.cronometer.android.gold',
    'FITTR: Fitness&Weigth Loss':'com.squats.fittr',
    'Macros - Kalorienzähler':'com.josmantek.macros',
    'Kalorienzähler Kalorien!':'org.digitalcure.ccnf.app',
    'Abnehmen ohne Diät':'ru.harmonicsoft.caloriecounter'
}

In [52]:
def scrape_reviews(app_id, sort, count):
    result, continuation_token = reviews(
        app_id,
        lang='en', # defaults to 'en'
        country='us', # defaults to 'us'
        sort=sort, # defaults to Sort.NEWESTs
        count=count, # defaults to 100
    )
    return result   

In [53]:
def detect_language(text):
    try:
        lang = detect(text)
    except Exception:
        lang = 'unknown'
    return lang 

In [54]:
reviews_df = pd.DataFrame()
for app_name, app_id in apps.items():
    for sort in [Sort.MOST_RELEVANT, Sort.NEWEST]:
        result = scrape_reviews(app_id, sort=sort, count=500)
        if len(result) == 0:
            continue
        app_review_df = pd.DataFrame(np.array(result),columns=['review'])
        app_review_df = app_review_df.join(pd.DataFrame(app_review_df.pop('review').to_list()))
        app_review_df['lang'] = app_review_df['content'].apply(lambda x: detect_language(x))
        app_review_df['app_name'] = app_name
        app_review_df['source'] = 'most_relevant' if sort is Sort.MOST_RELEVANT else 'newest'
        reviews_df = pd.concat([reviews_df, app_review_df])

In [71]:
reviews_df.to_csv('reviews.csv')